In [1]:
!pip install scikit-learn 
!pip install joblib
!pip install SpeechRecognition
!pip install pyttsx3
!pip install pyAudio
!pip install pywhatkit
!pip install pyJokes
!pip install wikipedia
!pip install pyowm
!pip install gtts
!pip install yt_dlp python-vlc
!pip install pywin32
!pip install pytube
!pip install pytube youtube-search-python
!pip install nltk
!pip install transformers
!pip install --upgrade spacy
!pip install --upgrade spacy-lookups-data
!python -m spacy download en_core_web_sm
!pip install spacy transformers

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.0/12.8 MB 330.3 kB/s eta 0:00:39
     --------------------------------------- 0.1/12.8 MB 438.9 kB/s eta 0:00:30
     --------------------------------------- 0.1/12.8 MB 598.8 kB/s eta 0:00:22
      -------------------------------------- 0.2/12.8 MB 700.2 kB/s eta 0:00:19
      -------------------------------------- 0.3/12.8 MB 983.0 kB/s eta 0:00:13
     - -------------------------------------- 0.4/12.8 MB 1.1 MB/s eta 0:00:12
     - -------------------------------------- 0.5/12.8 MB 1.2 MB/s eta 0:00:11
     - -------------------------------------- 0.5/12.8 MB 1.3 MB/s eta 0:00:10
     - -------------------------------------- 0.6/12.8 MB 1.4 MB/s eta 0:00:09
     -- ------------------------------------- 0.7/12.8 MB 1.5 MB/s eta 0:00:09
     -- ------------------------------------- 0.8/12.8 MB 1.4

In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier , AdaBoostClassifier
from sklearn.metrics import classification_report
from sklearn.utils import resample
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import cross_val_score, cross_validate

import seaborn as sns
import matplotlib.pyplot as plt
import joblib
import speech_recognition as sr
import pyttsx3
import pywhatkit
import datetime
import wikipedia
import pyjokes
import pywhatkit
import requests
import gtts
import threading
import yt_dlp
import vlc
import threading 
import pythoncom
from pytube import YouTube
import os
import re
import spacy

In [ ]:
#with open('command_intent.csv', 'r', encoding='ISO-8859-1') as file:
    #content = file.read()
#with open('command_intent_utf8.csv', 'w', encoding='utf-8') as file:
    #file.write(content)
df = pd.read_csv('command_intent_with_time.csv')


In [ ]:
df.head

In [ ]:
print(df['intent'].value_counts())

In [ ]:
df_up_sampled = pd.DataFrame()
for intent in df['intent'].unique():
    df_class = df[df['intent'] == intent]
    if len(df_class) < 42:
        df_class_up_sampled = resample(df_class, 
                                      replace=True,          # sample with replacement
                                      n_samples=42,  # match majority class size
                                      random_state=42)       # reproducible results
        df_up_sampled = pd.concat([df_up_sampled, df_class_up_sampled])
    else:
        df_up_sampled = pd.concat([df_up_sampled, df_class])
# Shuffle the upsampled dataframe
df_up_sampled = df_up_sampled.sample(frac=1, random_state=42).reset_index(drop=True)
# Check the new class distribution
print(df_up_sampled['intent'].value_counts())

In [ ]:
#assigning commands to variable x and intent to y for model 
x = df_up_sampled['command']
y = df_up_sampled['intent']

In [ ]:
#using labelencoder to encode intents
label_encoder = LabelEncoder()
label_encoder.fit_transform(y)
print("Classes:", label_encoder.classes_)

In [ ]:
#train test splitiing 
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2 , random_state = 42 , stratify =y)

In [ ]:
#introducing pipeline with vectorizer is count vectorizer and classif is randomforest
pipeline = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])
#cross validation to evaluate model performance 
cv_scores = cross_val_score(pipeline, x, y, cv=5, scoring='accuracy')
print(f"Cross-validation scores: {cv_scores}")
print(f"Mean cross-validation score: {cv_scores.mean():.4f}")
#model training 
pipeline.fit(x_train, y_train)
#prediction 
y_pred = pipeline.predict(y_test)
#print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

In [ ]:
y_pred = pipeline.predict(x_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

In [ ]:
joblib.dump(pipeline, 'intent_pipeline.pkl')  #saving model 

In [ ]:
# Load the pipeline
intent_pipeline = joblib.load('intent_pipeline.pkl')


listener = sr.Recognizer()
listener.energy_threshold = 4000
engine = pyttsx3.init()
voices = engine.getProperty('voices')
engine.setProperty('voice', voices[1].id)

player = None
is_playing = False
is_paused = False

def adjust_threshold(source):
    listener.adjust_for_ambient_noise(source, duration=3)

def engine_talk(text):
    engine.say(text)
    engine.runAndWait()

def weather(city):
    api_key = "142ba3ad77c737d4d7023e7381b9b27a"
    base_url = "http://api.openweathermap.org/data/2.5/weather?"
    complete_url = base_url + "appid=" + api_key + "&q=" + city
    response = requests.get(complete_url)
    x = response.json()
    if x["cod"] != "404":
        y = x["main"]
        current_temperature = y["temp"]
        current_temperature_celsius = current_temperature - 273.15  # Convert from Kelvin to Celsius
        return str(round(current_temperature_celsius, 2))
    else:
        return "City Not Found"

def play_song(song):
    global player
    global is_playing
    global is_paused
    pythoncom.CoInitialize()  # Initialize COM library
    player = vlc.MediaPlayer()  # Initialize player in the function
    ydl_opts = {
        'format': 'bestaudio/best',
        'noplaylist': True,
        'quiet': True,
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info_dict = ydl.extract_info(f"ytsearch:{song}", download=False)
        video_url = info_dict['entries'][0]['url']
        print(f"Playing URL: {video_url}")  # Debugging print
        player.set_mrl(video_url)
        player.play()
        is_playing = True
        is_paused = False
        print("Player should be playing now.")  # Debugging print

def pause_song():
    global is_playing
    global is_paused
    if player and is_playing:
        player.pause()
        is_playing = False
        is_paused = True
        print("Song paused.")  # Debugging print

def resume_song():
    global is_playing
    global is_paused
    if player and is_paused:
        player.play()
        is_playing = True
        is_paused = False
        print("Song resumed.")  # Debugging print

def stop_song():
    global is_playing
    global is_paused
    if player:
        player.stop()
        is_playing = False
        is_paused = False
        print("Song stopped.")  # Debugging print

def fetch_wikipedia_summary(name):
    try:
        # Fetch a summary of the given name from Wikipedia
        summary = wikipedia.summary(name, sentences=2)
        return summary
    except wikipedia.exceptions.DisambiguationError as e:
        # Handle the case where the name matches multiple topics
        engine_talk("The term is ambiguous. Did you mean one of the following? specify with what is")
        for option in e.options[:5]:  # Limit to first 5 options
            engine_talk(option)
        return ""
    except wikipedia.exceptions.PageError:
        # Handle the case where the page does not exist
        engine_talk("Sorry, I couldn't find any information on that topic.")
        return ""
    except Exception as e:
        # Handle any other exceptions
        engine_talk(f"Sorry, an error occurred: {str(e)}")
        return ""

def user_commands():
    with sr.Microphone() as source:
        adjust_threshold(source)
        print("Listening...")
        voice = listener.listen(source)
        try:
            command = listener.recognize_google(voice)
            command = command.lower()
            print(f"Recognized command: {command}")
            if 'lyra' in command:
                command = command.replace('lyra', '').strip()
            return command
        except sr.UnknownValueError:
            engine_talk("Sorry, I did not get that")
            return ""
        except sr.RequestError:
            engine_talk("Sorry, my speech service is down")
            return ""

def classify_intent(command):
    # Use the pipeline to predict the intent
    intent = intent_pipeline.predict([command])[0]
    return intent

nlp = spacy.load("en_core_web_sm")

def extract_city(command):
    print('the command got in extract_city',command)
    doc = nlp(command)
    for entity in doc.ents:
        if entity.label_ == 'GPE':
            print(entity.text)
            return entity.text
    return None

def run_lyra():
    engine_talk("I am your voice assistant Lyra. What can I do for you?")
    global is_playing
    global is_paused
    while True:
        command = user_commands()
        if command:
            intent = classify_intent(command)
            print(f"Predicted intent: {intent}")

            if intent == 'play_song':
                song = command.replace('play', '').strip()
                engine_talk('Playing ' + song)
                play_song(song)
            elif intent == 'pause_song':
                if is_playing:
                    pause_song()
                    engine_talk('Pausing the song')
            elif intent == 'resume_song':
                if is_paused:
                    resume_song()
                    engine_talk('Resuming the song')
            elif intent == 'stop_song':
                if is_playing or is_paused:
                    stop_song()
                    engine_talk('Stopping the song')
            elif intent == 'restart_song':
                if is_playing or is_paused:
                    stop_song()
                song = command.replace('restart', '').strip()
                play_song(song)
                engine_talk('Playing the song from the beginning')
            elif intent == 'fetch_info':
                name = command.replace('who is', '').replace('what is', '').strip()
                info = fetch_wikipedia_summary(name)
                print(info)
                engine_talk(info)
            elif intent == 'tell_joke':
                joke = pyjokes.get_joke()
                print(joke)
                engine_talk(joke)
            elif intent == 'time':
                time = datetime.datetime.now().strftime('%I:%M %p')
                engine_talk('The current time is ' + time)
                print('The current time is  ' + time)
            elif intent == 'get_weather':
                city = extract_city(command)
                if city:
                    print('City:', city)
                    weather_info = weather(city)
                    if weather_info == "City Not Found":
                        engine_talk("City not found")
                    else:
                        print(f'the current temeprature in {city} is {weather_info} degrees celsius')
                        engine_talk(f'The current temperature in {city} is {weather_info} degrees Celsius')
                else:
                    print("Sorry, I couldn't determine the city. Please ask again with a specific city name.")
                    engine_talk("Sorry, I couldn't determine the city. Please ask again with a specific city name.")
            elif intent == 'thank_you':
                print('You are welcome')
                engine_talk('You are welcome')
            elif intent == 'shut_down':
                print('goodbye!')
                engine_talk('Goodbye!')
                break
            else:
                print('I could not hear you properly')
                engine_talk('I could not hear you properly')

run_lyra()